In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
import constants as c
from requests.auth import HTTPBasicAuth
import utils
import requests
import csv
import pandas as pd

# Process Commit Data from BusinessLogic

Make sure that you have generated the file `commit_data.csv` in the data folder   
Run the notebook `commits.ipynb`

## Read Data

In [ ]:
def clean_authors(row):
    if row == 'rasmus.emil.skousen':
        return 'res'
    
    if row == 'frederik':
        return 'fcm'
    
    return row

In [ ]:
# Read data into pandas dataframe 
# The data is read from a CSV file, which is generated by the script `commits.py`
df = pd.read_csv(c.COMMIT_DATA_FILE_NAME)

# Convert the date column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Clean Authors (some people have multiple usernames/emails)
# rasmus.emil.skousen -> res
# frederik -> fcm
df['author'] = df['author'].apply(clean_authors)

## Summary

In [ ]:
# Get the max date in the dataset
max_date = df['date'].max()
min_date = df['date'].min()
unique_authers = df['author'].unique()

print("Max date: ", max_date.strftime("%Y-%m-%d"))
print("Min date: ", min_date.strftime("%Y-%m-%d"))

print()
print("Number of unique authors: ", len(unique_authers))
print("Unique authors: ", unique_authers)

In [ ]:
min_date + pd.DateOffset(months=2)

In [ ]:
df.drop_duplicates(inplace=True)
print("Original row count: ", df.shape[0])
# Only select rows where the date is min_date + 2 months

df = df[(df['date'] >= min_date + pd.DateOffset(months=2))]


## Grouped By Author

In [ ]:
df_commits_pr_author = df.groupby('author').size().reset_index(name='count').sort_values(by='count', ascending=False)
df_commits_pr_author.rename(columns={'count': 'commits'}, inplace=True)
df_commits_pr_author

In [ ]:
df_commits_pr_author["commits"].sum()

In [ ]:
df_grouped_by_author = df.groupby('author').sum(["adds","deletes", "changes"]).sort_values(by='changes', ascending=False)
df_grouped_by_author

In [ ]:
df_grouped_by_author_with_commits = df_grouped_by_author.join(df_commits_pr_author.set_index('author'), on='author')

In [ ]:
new_column_order = ['author', 'commits', 'adds', 'deletes', 'changes']
df_grouped_by_author_with_commits.reset_index()[new_column_order]


In [ ]:
df_grouped_by_author["total"] = df_grouped_by_author["adds"] + df_grouped_by_author["deletes"] + df_grouped_by_author["changes"]

In [ ]:
df_grouped_by_author.sort_values(by='total', ascending=False).reset_index()